# Integridad Referencial




In [23]:
-- Para dejar en blanco la base de datos PRUEBA ejecute este bloque 
-- Ejecute este bloque(colapsado) para usarla  
use master

IF EXISTS  
    ( 
        SELECT name FROM master.dbo.sysdatabases  
        WHERE name = 'Pruebas' 
    ) 
    ALTER DATABASE pruebas SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
    DROP DATABASE Pruebas 
GO 
    
CREATE DATABASE Pruebas 
GO 
use Pruebas


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:01.789

## Paso 1 ##
Sea una entidad *Empleado* que *trabaja* en un *Departamento*


In [24]:
CREATE TABLE Empleados
(
    Nombre nvarchar(30), 
    Apellido nvarchar(30),
    RFC CHAR(13), 
    CURP CHAR(18) not null primary key,
    NSS CHAR(20),
    Nacimiento DATE,
    Trabaja CHAR(3) --Lo agregamos para la relación trabaja
)

CREATE TABLE Departamentos
(
    idDepartamento CHAR(3),
    nombre NVARCHAR(30),
    descripcion NVARCHAR(200),
    mision NVARCHAR(200)
)

INSERT into Departamentos (idDepartamento, nombre) 
    values  ('VEN', 'Ventas' ), 
            ('MKT', 'Mercadotecnia'),
            ('PRO', 'Producción')

INSERT into empleados (CURP, Apellido, Nombre, Trabaja)
    VALUES ( 'LOEJ880305RESHS000', 'Lopez Espericueta', 'Jose', 'MKT')
INSERT into empleados (CURP, Apellido, Nombre, Trabaja)
    VALUES ( 'LAEJ880305RESHS000', 'Larrea Espericueta', 'Jose', 'MKT')
INSERT into empleados (CURP, Apellido, Nombre, Trabaja)
    VALUES ( 'LOTJ880305RESHS000', 'Lopez Tarriba', 'Jose', 'PRO')
INSERT into empleados (CURP, Apellido, Nombre, Trabaja)
    VALUES ( 'LOEO880305RESHS000', 'Lopez Omar', 'Jose', 'VEN')            

(3 rows affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.149

Al momento de asignar un empleado a un departamento ¿ puede hacerse aunque que este no exista ?

Respuesta: 
Si, ya que no hay una verificacion de si el departamento existe dentro de la tabla Departamentos. Necesitaría la conexión entre tablas por medio de una Foreign Key desde el atributo "trabaja" de empleados hacia el atributo "idDepartamento" de la tabla Departamentos.

In [25]:
INSERT into empleados (CURP, Apellido, Nombre, Trabaja)
    VALUES ( 'VICJ880305RESHS000', 'Vizcarra Camacho', 'Jaime', 'TIC')
SELECT * from Empleados


(1 row affected)

(5 rows affected)

Total execution time: 00:00:00.119

Nombre,Apellido,RFC,CURP,NSS,Nacimiento,Trabaja
Jose,Larrea Espericueta,NULL,LAEJ880305RESHS000,NULL,NULL,MKT
Jose,Lopez Espericueta,NULL,LOEJ880305RESHS000,NULL,NULL,MKT
Jose,Lopez Omar,NULL,LOEO880305RESHS000,NULL,NULL,VEN
Jose,Lopez Tarriba,NULL,LOTJ880305RESHS000,NULL,NULL,PRO
Jaime,Vizcarra Camacho,NULL,VICJ880305RESHS000,NULL,NULL,TIC


## Paso 2##

La integridad referencial evita esto. Agreguemos una restriccion de llave foránea a la columna `Trabaja` de la tabla `Empleados`, para definirlo usamos la clausula `CONSTRAINT constraint_name FOREIGN KEY` que puede pertenecer a la sentencia `CREATE TABLE` o a `ALTER TABLE`
```
[ FOREIGN KEY ]
        REFERENCES [ schema_name . ] referenced_table_name [ ( ref_column ) ]
        [ ON DELETE { NO ACTION | CASCADE | SET NULL | SET DEFAULT } ]
        [ ON UPDATE { NO ACTION | CASCADE | SET NULL | SET DEFAULT } ]
        [ NOT FOR REPLICATION ]
```

In [26]:
-- Primero eliminamos la referencia al departamento inexistente
Update Empleados    
    set Trabaja = null
    where Trabaja = 'TIC'

(1 row affected)

Total execution time: 00:00:00.028

In [27]:
GO
ALTER TABLE Departamentos alter COLUMN idDepartamento char(3) NOT NULL

GO
ALTER TABLE Departamentos ADD CONSTRAINT pk_depto PRIMARY KEY(idDepartamento)

GO
ALTER TABLE empleados ADD CONSTRAINT fk_depto FOREIGN KEY(trabaja) REFERENCES Departamentos(idDepartamento)

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.096

Intente agregar nuevamente un empleado con un departamento inexistente

In [28]:
INSERT into empleados (CURP, Apellido, Nombre, Trabaja)
    VALUES ( 'VICJ880305RESHS000', 'Vizcarra Camacho', 'Jaime', 'TIC')
--Valores
SELECT * from Empleados


: Msg 2627, Level 14, State 1, Line 1
Violation of PRIMARY KEY constraint 'PK__Empleado__F46C4CBE38187562'. Cannot insert duplicate key in object 'dbo.Empleados'. The duplicate key value is (VICJ880305RESHS000).

The statement has been terminated.

(5 rows affected)

Total execution time: 00:00:00.186

Nombre,Apellido,RFC,CURP,NSS,Nacimiento,Trabaja
Jose,Larrea Espericueta,NULL,LAEJ880305RESHS000,NULL,NULL,MKT
Jose,Lopez Espericueta,NULL,LOEJ880305RESHS000,NULL,NULL,MKT
Jose,Lopez Omar,NULL,LOEO880305RESHS000,NULL,NULL,VEN
Jose,Lopez Tarriba,NULL,LOTJ880305RESHS000,NULL,NULL,PRO
Jaime,Vizcarra Camacho,NULL,VICJ880305RESHS000,NULL,NULL,NULL


## Paso 3: Eliminando referidos##

Asigne los empleados sin departamento al de *Ventas*

In [32]:

-- UPDATE

UPDATE empleados SET Trabaja='VEN' WHERE Trabaja IS NULL

SELECT * FROM Empleados

(1 row affected)

(5 rows affected)

Total execution time: 00:00:00.070

Nombre,Apellido,RFC,CURP,NSS,Nacimiento,Trabaja
Jose,Larrea Espericueta,NULL,LAEJ880305RESHS000,NULL,NULL,MKT
Jose,Lopez Espericueta,NULL,LOEJ880305RESHS000,NULL,NULL,MKT
Jose,Lopez Omar,NULL,LOEO880305RESHS000,NULL,NULL,VEN
Jose,Lopez Tarriba,NULL,LOTJ880305RESHS000,NULL,NULL,PRO
Jaime,Vizcarra Camacho,NULL,VICJ880305RESHS000,NULL,NULL,VEN


¿Que sucede si quiero eliminar el departamento *Ventas*?

**Respuesta**

Muestra un error ya que se está tratando de eliminar un registro que contiene una Primary Key que hace referencia a la restriccion fk_depto, lo que significa que hay registros de empleados que trabajan en dicho departamentos. Por lo tanto, no se puede borrar hasta que se eliminen o actualizen dichos empleados.

In [34]:
DELETE FROM Departamentos WHERE idDepartamento='VEN'

: Msg 547, Level 16, State 0, Line 1
The DELETE statement conflicted with the REFERENCE constraint "fk_depto". The conflict occurred in database "Pruebas", table "dbo.Empleados", column 'Trabaja'.

The statement has been terminated.

Total execution time: 00:00:00.032

Modifiquemos la restricción para que ahora al eliminar un *Departamento* elimine también a los empleados asignados a el

In [35]:
--Eliminar la restricción de llave foránea
ALTER TABLE empleados DROP CONSTRAINT fk_depto

--No se olvide del GO
GO
-- Agregamos la clausula ON DELETE y ON UPDATE
ALTER TABLE Empleados ADD CONSTRAINT  fk_depto FOREIGN KEY (Trabaja) REFERENCES Departamentos(idDepartamento)
ON DELETE CASCADE ON UPDATE CASCADE


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.069

In [37]:
DELETE FROM DEPARTAMENTOS WHERE idDEPARTAMENTO ='VEN'
SELECT  * FROM DEPARTAMENTOS

(0 rows affected)

(2 rows affected)

Total execution time: 00:00:00.098

idDepartamento,nombre,descripcion,mision
MKT,Mercadotecnia,NULL,NULL
PRO,Producción,NULL,NULL


¿Que sucede ahora si intenta eliminar el departamento *Ventas* ? 

**Respuesta**

Se eliminará tanto el registro del departamento "Ventas", así como todos los registros de Empleados que Trabajaban y hacian referencia al mismo.




## Paso 4 ##

Modifiquemos el comportamiento de la restricción para que al eliminar un departamento los empleados queden sin uno asignado ( `SET NULL`)
¿Es necesario eliminar la restricción y volverla a crear con el nuevo comportamiento ?

**Respuesta**


In [39]:

ALTER TABLE Empleados DROP CONSTRAINT fk_depto

ALTER TABLE Empleados ADD CONSTRAINT fk_depto FOREIGN KEY (Trabaja) REFERENCES Departamentos(idDepartamento)
ON DELETE SET NULL ON UPDATE SET NULL


Commands completed successfully.

Total execution time: 00:00:00.029

Pruebe el comportamiento eliminando un departamento y muestre como queda el empleado

In [41]:
DELETE FROM DEPARTAMENTOS WHERE idDepartamento='MKT'
SELECT * FROM Empleados

(0 rows affected)

(3 rows affected)

Total execution time: 00:00:00.057

Nombre,Apellido,RFC,CURP,NSS,Nacimiento,Trabaja
Jose,Larrea Espericueta,NULL,LAEJ880305RESHS000,NULL,NULL,NULL
Jose,Lopez Espericueta,NULL,LOEJ880305RESHS000,NULL,NULL,NULL
Jose,Lopez Tarriba,NULL,LOTJ880305RESHS000,NULL,NULL,PRO


## Comentarios a la práctica##
<Su comentario aquí>